In [ ]:
pip install selenium

In [3]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By

In [4]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    
    # Only views tweets with impression counts
    imp_cnt = []
    views = card.find_element(By.XPATH, './/a[@class="css-4rbku5 css-18t94o4 css-1dbjc4n r-1loqt21 r-1777fci r-bt1l66 r-1ny4l3l r-bztko3 r-lrvibr" and @aria-label]').get_attribute("aria-label")
    imp_cnt = re.search(r'\d+', views)
    if not isinstance(imp_cnt, re.Match):
        return
    imp_cnt = imp_cnt.group()
    
    username = card.find_element(By.XPATH, './/span').text
    try:
        handle = card.find_element(By.XPATH, './/span[contains(text(), "@")]').text
    except NoSuchElementException:
        return

    try:
        postdate = card.find_element(By.XPATH, './/time').get_attribute('datetime')
    except NoSuchElementException:
        return

    comment = card.find_element(By.XPATH, './/div[2]/div[2]/div[1]').text
    responding = card.find_element(By.XPATH, './/div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element(By.XPATH, './/div[@data-testid="reply"]').text or '0'
    retweet_cnt = card.find_element(By.XPATH, './/div[@data-testid="retweet"]').get_attribute("aria-label")
    retweet_cnt = re.search(r'\d+', retweet_cnt).group()
    like_cnt = card.find_element(By.XPATH, './/div[@data-testid="like"]').get_attribute("aria-label")
    like_cnt = re.search(r'\d+', like_cnt).group()

    # Extract image URLs (if available)
    image_urls = []
    try:
        images = card.find_elements(By.XPATH, './/img[@src]')
        for img in images:
            image_urls.append(img.get_attribute('src'))
    except NoSuchElementException:
        pass
    image_urls_str = ', '.join(image_urls)
    
    # Calculate engagement score
    engagement_sum = float(retweet_cnt or 0) + float(like_cnt or 0)
    engagement_score = str("%.2f" % ((float(engagement_sum or 0) / float(imp_cnt or 1)) * 100.0))
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt, imp_cnt, engagement_score, image_urls_str)
    return tweet

In [17]:
search_term = input('search term: ')

# create instance of web driver
driver = Chrome()

# navigate to login screen
driver.get('https://twitter.com/search')
# driver.maximize_window()
sleep(4)

username = driver.find_element(By.XPATH, "//input")
username.send_keys('VirtualWorld454')
username.send_keys(Keys.RETURN)
sleep(2)

password = driver.find_element(By.XPATH, "//input[@name='password']")
password.send_keys('HC1Project')
password.send_keys(Keys.RETURN)
sleep(3)

# find search input and search for term
search_input = driver.find_element(By.XPATH, '//input')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(2)

# navigate to historical 'latest' tab
driver.find_element(By.LINK_TEXT, 'Latest').click()

search term:  #lotr


In [18]:
# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)

    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1

            if scroll_attempt >= 4: # number of scroll attempts
                scrolling = False
                break
            else:
                sleep(4) # seconds before attempting another scroll
        else:
            last_position = curr_position
            break

# close the web driver
# driver.close()

Saving tweet data

In [20]:
with open('turkcell_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets', 'Views', 'Engagement Score', 'ImageURLs']

    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)